<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Carga librerías</span>

In [7]:
import ultralytics
import sklearn
from ultralytics import YOLO
from ultralytics import settings
import os
import torch
import subprocess
import gdown
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import shutil
from sklearn.model_selection import train_test_split
import yaml
from threading import Thread
from queue import Queue
from PIL import Image
import re
from pathlib import Path
import cv2
import tempfile

<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Carga rutas</span>

In [8]:
current_folder= os.getcwd() # Obtener ubicación actual
main_folder= os.path.abspath(os.path.join(current_folder, "../../../")) # Carpeta main del proyecto
main_staticos= os.path.abspath(os.path.join(main_folder))+'\\dist' # Carpeta main de los archivos estaticos
main_production= os.path.abspath(main_folder) +'\\src' # Carpeta main de los archivos listos para producción 
dir_prueba= os.path.abspath(main_folder) + "\\src\\assets_marks"  # Carpeta de imagenes prueba para el proceso de predecir

training_structure="MARCAS" # Indica el nombre de la carpeta que tiene los archivos para el entrenamiento (image, json)
dir_train= os.path.abspath(main_staticos) + f'\\{training_structure}' 

<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Configuración del data | Transición de las marcas en un formato segmentado hacia una estructura global.</span>

In [24]:
# Función para mover archivos de carpetas individuales a una carpeta global marcas, utilice esta función solo si tiene las marcas ordenadas por carpeta.

def process_brand_data(data_directory):
    destination_dir = os.path.abspath(os.path.join(data_directory, "../")) + "\\global_marks"
    print(destination_dir)
    
    if not os.path.exists(destination_dir): 
        os.makedirs(destination_dir)
        
        for (root, dirs, files) in os.walk(data_directory):
            if '______README______' in dirs:
                dirs.remove('______README______')      
            for file_name in files:
                if file_name.endswith(('.jpg', '.png', '.json')):
                    source_path = os.path.join(root, file_name)
                    destination_path = os.path.join(destination_dir, file_name)
                    print(f'copying from {source_path} to {destination_path}')
                    try:
                        shutil.copy(source_path, destination_path)  # Use shutil.move if you only want to move, not create a copy.
                    except Exception as e:
                        print(f'could not move the file {source_path} due to {e}')
    
    return destination_dir

dir_train=process_brand_data(dir_train)

c:\Users\matrix\pruebayolo\proyecto_yolo\dist\marcas_global


<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Instalación de herramientas de etiquetado</span>

In [35]:
# Función para instalar los paquetes
def package_verification(package):
    try:
        output = os.popen('pip list').read()
        return f'{package}' in output
    except Exception as e:
        print(f'Error en la verificación: {e}')
        return False

def package_installation(package_name):
    if package_verification(package_name):
        print(f"El paquete '{package_name}' ya está instalado. ")
    else:
        print('Instalando el paquete... ')
        try:
            os.system(f'pip install {package_name}')
            print(f'libreria {package_name} instalada correctamente! ')
        except Exception as e:
            print(f'Error en la instalación de {package_name}: {e}')

In [36]:
package_installation('labelme') # Instalación de la herramienta de etiquetado Labelme

El paquete 'labelme' ya está instalado. 


In [25]:
# Abrir la herramienta para el proceso de etiquetado 
try:
    subprocess.check_call(["labelme"]) 
except Exception as e:
    print(f'No fue posible abrir labelme: {e}')

In [37]:
package_installation('labelme2yolo') # Instalación de la herramienta de conversión para el formato JSON de labelme a formato texto requerido por los modelos de detección de objetos YOLO.

El paquete 'labelme2yolo' ya está instalado. 


<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Configuración de rutas de entrenamiento | Separación aleatoria del data</span>

In [ ]:
# Función para separar la carpeta global de todas las marcas en un 80% entrenamiento y 20% validación

def file_separation(orig_folder):
    file_list = os.listdir(orig_folder)
    train, val = train_test_split(file_list, test_size=0.2, random_state=100)
    if not os.path.exists("train"):
        train_path = os.path.join(orig_folder, "train")
        os.makedirs(train_path, exist_ok=True)
    move_files(train, orig_folder, train_path)
    
    if not os.path.exists("val"):
        val_path = os.path.join(orig_folder, "val")
        os.makedirs(val_path, exist_ok=True)
    move_files(val, orig_folder, val_path)

def move_files(file_list, orig_folder, dest_folder):
    for file in file_list:
        if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
            orig_image_path = os.path.join(orig_folder, file)
            json_name = os.path.splitext(file)[0] + ".json"
            
            orig_json_path = os.path.join(orig_folder, json_name)
            dest_image_path = os.path.join(dest_folder, file)
            dest_json_path = os.path.join(dest_folder, json_name)
            
            shutil.copy(orig_image_path, dest_image_path)
            shutil.copy(orig_json_path, dest_json_path)

file_separation(dir_train)

<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Conversión a formato YOLO</span>

In [26]:
# Función para convertir la carpeta de entrenamiento y validación en formato YOLO

def convert_to_yolo_format(training_directory, validation_directory):
    try:
        subprocess.check_call(["labelme2yolo", "--json_dir", training_directory])
        print(f'Conversión completada para la carpeta {training_directory}')
        subprocess.check_call(["labelme2yolo", "--json_dir", validation_directory])
        print(f'Conversión completada para la carpeta {validation_directory}')
    except subprocess.CalledProcessError as e:
        print(f'Error al ejecutar función labelme2yolo: {e}')
        
#chevrolet_toyota_train= "C:/Users/matrix/pruebayolo/proyecto_yolo/dist/entrenamiento_final/lala/train"
#chevrolet_toyota_val= "C:/Users/matrix/pruebayolo/proyecto_yolo/dist/data_chevrolet_toyota/lala/val"

train_path= os.path.abspath(dir_train) + "\\train"
val_path= os.path.abspath(dir_train) + "\\val"

convert_to_yolo_format(train_path, val_path)
#convert_to_yolo_format(chevrolet_toyota_train, chevrolet_toyota_val)


Conversión completada para la carpeta C:/Users/matrix/pruebayolo/proyecto_yolo/dist/entrenamiento_final/lala/train
Conversión completada para la carpeta C:/Users/matrix/pruebayolo/proyecto_yolo/dist/entrenamiento_final/lala/val


In [28]:
# Organización de la estructura del repositorio YOLODataset (resultado de la conversión .json a formato YOLO) creado por defecto

def reorganization(path):
    try:
        new_path = os.path.abspath(os.path.join(path, "YOLODataset"))
        main_folders = os.listdir(new_path)
        for folder_instance in main_folders:
            secondary_folder_path = os.path.join(new_path, folder_instance)
            if os.path.isdir(secondary_folder_path):
                secondary_folders = os.listdir(secondary_folder_path)
                for secondary_folder in secondary_folders:
                    subfolder_secondary_path = os.path.join(secondary_folder_path, secondary_folder)
                    try:
                        if os.listdir(subfolder_secondary_path) != []:
                            for file in os.listdir(subfolder_secondary_path):
                                orig_path = os.path.join(subfolder_secondary_path, file)
                                dest_path = os.path.abspath(os.path.join(subfolder_secondary_path, "../"))
                                new_dest_path = os.path.join(dest_path, file)
                                shutil.move(orig_path, new_dest_path)
                            os.rmdir(subfolder_secondary_path)
                        else:
                            os.rmdir(subfolder_secondary_path)
                    except NotADirectoryError as e:
                        print(f'Error: the element is not a folder: {e}')
        return new_path
    except FileNotFoundError as e:
        print(f'Error: {e}')

yolo_train_data= reorganization(train_path)
yolo_val_data= reorganization(val_path)  

In [29]:
# Función para mover las carpetas yolo 

def acomodar_rutas_dataset(main_staticos, dataset_train, dataset_val):
    try:
        if not os.path.exists("data_marks"):
            try: 
                path_brands= os.path.join(main_staticos, "data_marks")
                os.makedirs(path_brands, exist_ok=True)
            
                shutil.move(dataset_train, os.path.join(path_brands, "YOLODataset_train"))
                shutil.move(dataset_val, os.path.join(path_brands, "YOLODataset_val"))
                print(f'Carpetas movidas exitosamente a {path_brands} ')
            except OSError as e:
                print(f'Error al intentar crear la carpeta en {path_brands}: {e}')
        return path_brands  
    except TypeError as e:
        print(f'Tipo de dato incorrecto para el argumento: {e}')

brands_route= acomodar_rutas_dataset(main_staticos, yolo_train_data, yolo_val_data)

Carpetas movidas exitosamente a c:\Users\matrix\pruebayolo\proyecto_yolo\dist\data_marcas 


In [30]:
# Set de funciones para ordenar la estructura de los dataset individuales.

def arrange_dataset_paths(train, val):
    destination = os.path.abspath(os.path.join(train, "../../"))
    shutil.move(train, destination)
    if "dataset.yaml" in os.listdir(os.path.join(val, "../")):
        os.remove(val)
    return destination

# Función para reescribir el dataset 
def rewrite_dataset(dir_train_converted, combined_list):
    with open(dir_train_converted, 'w') as new_dataset:
        new_dataset.write(
            f'train: {os.path.abspath(os.path.join(dir_train_converted, "../"))}\n'
            f'val: {os.path.abspath(os.path.join(dir_train_converted, "../../"))}\\YOLODataset_val\n'
            'test: \n'
            f'nc: {len(combined_list)}\n'
            f'names: {combined_list}\n'
        )
        
# Función para combinar los dos dataset generados
def total_labels(list1, list2, update_dataset_train):
    set1=set(list1)
    set2=set(list2)
    union_without_repeating = set1.union(set2)
    combined_list = list(union_without_repeating)
    rewrite_dataset(update_dataset_train, combined_list)


# Función para extraer los nombres de las etiquetas de los dos dataset separados

def extract_labels(main_route):
    try:
        route_dataset_train= os.path.abspath(main_route) + "\\YOLODataset_train\\dataset.yaml"
        route_dataset_val= os.path.abspath(main_route) + "\\YOLODataset_val\\dataset.yaml"
        
        with open(route_dataset_train, 'r') as file_train:
            data_train = yaml.safe_load(file_train)
            labels_train = data_train['names']
        with open(route_dataset_val,'r') as file_val:
            data_val= yaml.safe_load(file_val)
            labels_val= data_val['names']
        total_labels(labels_train, labels_val, route_dataset_train)
        
        return route_dataset_train, route_dataset_val
    except TypeError as e:
        print(f'Error: {e}')

try:
    train, val = extract_labels(brands_route)
    arrange_dataset_paths(train, val)
except TypeError as e:
    print(f'Error: {e}')

<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Ubicación de la Carpeta Preparada para el Entrenamiento</span>

In [24]:
# Buscar la carpeta que contiene la configuración completa para el entrenamiento
#data_train_marks= os.path.abspath(main_staticos) + "\\data_marks\\dataset.yaml" #Direccionamiento completo de todas las marcas.
data_train_marks= os.path.abspath(main_staticos) + "\\data_chevrolet_toyota\\dataset.yaml" # Direccionamiento de prueba para dos marcas debido a falta de datos.

<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Configuración de las carpetas predeterminadas de YOLO</span>

In [16]:
# Modificación de las rutas de las carpetas predeterminadas de ultralytics YOLO

def set_config_ultralytics(main):
    settings.update({'datasets_dir': f'{main}\\dist', 'weights_dir': f'{main}\\src', 'runs_dir': f'{main}\\src\\runs'})
set_config_ultralytics(main_folder)

<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Selección versión de modelo YOLOv8</span>

<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Modelo original</span>

### Tabla de variantes en la Segmentación de instancias con YOLOv8

| Modelo      | Nombres de archivo                                      | Identificador |
|-------------|-----------------------------------------------------|------------|
| YOLOv8-seg  | yolov8n-seg.pt yolov8s-seg.pt yolov8m-seg.pt yolov8l-seg.pt yolov8x-seg.pt | n, s, m, l, x |

In [ ]:
package_installation('gdown')

In [10]:
def get_model_version():
    available_versions= ['n', 's', 'm', 'l', 'x'] # yolov8n.pt , yolov8s.pt , yolov8m.pt , yolov8l.pt , yolov8x.pt
    control= True
    while control:
        version_selection= input("Ingrese la versión del modelo que desea utilizar, entre las disponibles están: n, s, m, l, x. Advertencia: Si no desea instalar ninguna ingrese la palabra 'salir'! ") 
        if version_selection in available_versions:
            id_version= f'yolov8{version_selection}-seg.pt' 
            return id_version
        elif version_selection == "salir":
            control=False
            return None                                                         
        else:
            print('Version de modelo inexistente, ingrese nuevamente! ')
     
def model_installation(url, version_name):
    try:
        path_save = os.path.abspath(main_folder) + "\\dist\\model_version\\" + version_name
        if os.path.exists(path_save):
            print(f'Se está utilizando la versión "{version_name}" del modelo. ')
        else:
            url_installation = f'{url}/{version_name}'
            gdown.download(url_installation, path_save, quiet=False)
            print('Versión instalada correctamente!')
    except (NameError, UnboundLocalError) as e:
        print(f'Error en la instalación de la versión del modelo: {e}')
    return path_save

model_url= 'https://github.com/ultralytics/assets/releases/download/v0.0.0' 
model_version= get_model_version()

if model_version is not None:
    model_orig= model_installation(model_url, model_version) 
else:
    print('Ha salido con exito! ')

Se está utilizando la versión "yolov8x-seg.pt" del modelo. 


<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Función selección modelos pre-entrenados</span>

In [11]:
# Función para obtener el más reciente entrenamiento

def get_last_training(dir_src, det_type):
    try:
        yolo_model_folders= os.listdir(dir_src)
        for folder_name in yolo_model_folders:
            if folder_name == det_type:
                base_path = os.path.abspath(os.path.join(dir_src, folder_name))
                files= os.listdir(base_path)
                last_experiment = sorted(files)[-1]
                model= os.path.join(base_path, last_experiment, "weights", "best.pt")
                print(f'Los pesos del experimento que se van a utilizar son {model}')
                return model
        else:
            print(f'No existe ningún experimento de entrenamiento en la carpeta {dir_src}')
            return None
    except FileNotFoundError as e:
        print(f'Error: {e}')

<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Modelo entrenado</span>

In [14]:
# Llamado para obtener el último entrenamiento del modelo marcas 

version_marcas= "srcv5" # Nombre de la carpeta donde se encuentra el modelo pre-entrenado a usar
detection_type_marcas= "segmentation" # Establece el modo de detecciones de Yolo (deteccion o segmentación)
dir_model_version_marcas=os.path.abspath(os.path.join(main_production, version_marcas)+ "\\model_marks")
model_marcas= get_last_training(dir_model_version_marcas, detection_type_marcas) 

Los pesos del experimento que se van a utilizar son c:\Users\matrix\pruebayolo\proyecto_yolo\src\srcv5\model_marks\segmentation\training_marks\weights\best.pt


<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Instalación de recursos</span>

In [ ]:
package_installation('ultralytics') # Instalación de la biblioteca Ultralytics
#ultralytics.checks()

In [ ]:
#Función para determinar si tiene GPU Cuda para instalación de PYTorch
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'CUDA: {torch.cuda.get_device_name(0)}')
else:
    try:
        command = 'wmic path win32_videocontroller get caption'
        device = subprocess.check_output(command, shell=True, universal_newlines=True)
        print(f'Tarjeta Grafica: {device.strip()}.\n \nAdvertencia: Debe utilizar la CPU para instalación de PyTorch! ')
    except subprocess.CalledProcessError as e:
        print(f'Error al ejecutar el comando: {e}')

<span style="font-family: 'Bahnschrift Light'; font-size: 18px">Dirección para instalar mediante el comando la versión de PyTorch en función de los requerimientos computacionales: </span>[Versión PyTorch](https://pytorch.org/get-started/locally/)

In [ ]:
command_pytorch = 'pip3 install torch torchvision torchaudio' # Comando de instalación según la version personalizada a sus requerimientos computacionales 
substrings = command_pytorch.split(" ")
try:
    subprocess.check_call(substrings)
except subprocess.CalledProcessError as e:
    print(f'Error al instalar la version de PyTorch: {e}')

<span style="font-family: 'Bahnschrift Light'; font-size: 40px;">ENTRENAMIENTO</span>


In [6]:
PROJECT='segmentation' # Permite asignar un nombre al directorio de inicio que contendrá los experimentos de segmentación de objetos, y debe estar entre comillas; se recomienda no utilsizar espacios en el nombre.
NAME='training_marks' # El nombre del experimento entrenamiento para segmentación de instancias debe ir entre comillas. Evite el uso de espacios al nombrar las carpetas; en su lugar, utilice algún formato de nombres como camelCase, snake_case o PascalCase.
TASK='segment' # Define la tarea principal que desea realizar con el modelo YOLO v8; en este caso, la segmentación implica el uso de máscaras para identificar objetos individuales en una imagen y segmentarlos del resto de la imagen.
IMGSZ=640 # Establezca las dimensiones en píxeles de la imagen de entrada. Puede especificarlo como un número entero, como imgsz a 640 para obtener un cuadrado perfecto, o como una tupla, como imgsz=(640,480) para establecer dimensiones específicas de ancho y alto. Se recomienda ajustar este valor según el tamaño del objeto que desea segmentar. Para la segmentación de objetos pequeños, se recomienda aumentar el valor a más de 640 píxeles para obtener una resolución más alta.
DATA= data_train_marks #  Le permite indicar la ruta al archivo que contiene los metadatos que utilizara el modelo segmentación de objetos y su configuración en formato YAML. Si especifica el valor data=None, el conjunto de datos coco128-seg.yaml se utiliza de forma predeterminada; de lo contrario, escriba la ruta al archivo YAML entre comillas utilizando barras diagonales (/) en lugar de barras invertidas (\).
EPOCHS=45 # Establezca el número de épocas del modelo YOLO v8 en la tarea de segmentación. Este valor representa el número total de iteraciones en todo el conjunto de datos de entrenamiento. Se recomienda experimentar con este parámetro dependiendo de la cantidad de imágenes disponibles. Si tiene un conjunto de datos grande, considere aumentar este valor por encima de 30 para obtener mejores resultados. Por otro lado, establecer epochs=None hará que el modelo continúe entrenándose hasta que la pérdida de validación deje de mejorar.
BATCH=-1 # Define la cantidad de imágenes procesadas simultáneamente en una iteración del modelo YOLO v8 en la segmentación. El valor predeterminado es 16; se recomienda establecerlo en -1 para aprovechar AutoBatch, que ajusta automáticamente el tamaño del lote para optimizar el rendimiento, evitar problemas de memoria y maximizar la eficiencia del entrenamiento. Si desea personalizarlo, exprese el valor del parámetro como un número entero.
OPTIMIZER='auto' # Define el algoritmo de optimización para el modelo de segmentación YOLO. Su elección ajusta los pesos del modelo durante el entrenamiento y es crucial para la velocidad y rendimiento. Puede tomar valores como 'SGD', 'Adam', 'Adamax', 'AdamW', 'NAdam', 'RAdam', 'RMSProp' y 'auto', este último selecciona automáticamente el optimizador más adecuado a la tarea segmentación de objetos.
WORKERS=1 # Especifica la cantidad de subprocesos o núcleos utilizados para cargar datos en el modelo segmentación YOLO. Se recomienda ajustar la cantidad de subprocesos a la cantidad de núcleos de CPU disponibles en el sistema.
DEVICE= 'cpu' # Especifica el dispositivo de ejecución para la versión del modelo yolov8 en la operación de segmentación. Puede seleccionar entre CPU o GPU. Si no dispone de una GPU con Cuda, se recomienda utilizar la CPU mediante el parámetro device='cpu'. En caso de contar con Cuda, puede especificar una GPU con device='cuda:0'; el número representa el identificador de la GPU disponible en el sistema. También es posible utilizar múltiples GPUs mediante device='cuda:0,1,2'.
PLOTS=True # Utilice valores booleanos (Verdadero o Falso) para controlar la generación de gráficos que permite visualizar y monitorear la pérdida y la precisión durante el entrenamiento de segmentación de objetos. Establecer plots=True activara la función; si desea desactivarla, establezca el valor del hiperparámetro en False.
SAVE=False # Cuando se establece en True, el modelo guarda puntos de control periódicamente durante el entrenamiento de segmentación. Se recomienda tener cuidado al establecer este valor en Verdadero, ya que está relacionado con el hiperparámetro SAVE_PERIOD; si establece el valor del hiperparametro a False, la función Save_Period se desactivará.
SAVE_PERIOD=-1 # Se utiliza para especificar con qué frecuencia se guardan los puntos de control durante el entrenamiento de segmentación. Si se establece en un valor mayor que 0, el modelo guardará puntos de control cada número especificado de épocas. Sin embargo, si save_period se establece en -1, significa que la función esta deshabilitada.
PATIENCE=45 # Representa el número esperado de épocas durante el entrenamiento de segmentación. Si no se observa mejora en el conjunto de validación dentro de un período específico, se detiene el proceso. Esta técnica de parada temprana se utiliza para evitar el sobreajuste del modelo. Se recomienda ajustar este hiperparámetro en función de la duración esperada del entrenamiento.
VERBOSE=False # Se utiliza para controlar el número de impresiones durante la ejecución del entrenamiento de segmentación. Para suprimir la salida de información básica únicamente, debe establecer el valor del hiperparámetro en False, pero si desea una salida de progreso más detallada, establezca el valor en True.
RECT= False # Habilita la formación rectangular en cada lote, redimensionando las imágenes para que todas tengan la misma forma rectangular. Puedes establecerlo en True si tu conjunto de datos es extenso y deseas acelerar el tiempo de entrenamiento en la segmentación de instancias. De lo contrario, si se establece en False el modelo se entrena en el orden normal procesando todos los datos de un lote antes de pasar al siguiente lote.
COS_LR=False # Reemplaza el decaimiento escalonado predeterminado de YOLOv8, que reduce la tasa de aprendizaje en ciertas épocas, con el decaimiento escalonado cos_lr. Este ajusta la tasa de aprendizaje según las épocas restantes y la tasa de aprendizaje inicial, proporcionando una disminución más suave. Establezca este hiperparámetro en True para una reducción gradual de la tasa de aprendizaje, de lo contrario establezca en False.
FRACTION= 1.0 # Controla la fracción del conjunto de datos que se utilizara para el entrenamiento de segmentación. Debes establecer este parámetro entre 0.0 y 1.0. Por defecto, cuando es 1.0, se emplea el 100% de las imágenes disponibles en el conjunto de datos.
EXIST_OK=False # Controla la sobreescritura de un experimento de segmentación existente. Cuando se establece en False, el sistema no sobrescribirá, en su lugar devolverá una ruta incrementada. Esto es útil para prevenir la sobreescritura accidental de experimentos anteriores. Para activar la función, asigna el valor True.
OVERLAP_MASK=True # Determina si las máscaras que representan áreas de interés en la imagen deben superponerse. Establecer el valor del hiperparametro a True permite que estas máscaras compartan áreas, lo que significa que los límites de los objetos en la imagen puede coincidir parcialmente. Esta opción ahorra memoria, acelera el entrenamiento, y es especialmente útil para grandes conjuntos de datos o modelos complejos.
MASK_RATIO=1 # Configure la reducción de muestreo para la máscara de segmentación. Se sugiere fijar el valor en el rango de 1 para entrenamiento en resolución nativa, ideal para aplicaciones que demandan alta precisión, hasta 4 para una reducción en un factor de 4, acelerando el entrenamiento con una precisión aceptable y ahorrando memoria. Este hiperparámetro solo acepta valores enteros.

def train(model_train):
    try:
        model_train.train(project=PROJECT,name=NAME, task=TASK, data=DATA, imgsz=IMGSZ, epochs=EPOCHS, batch=BATCH, workers=WORKERS, device=DEVICE, plots=PLOTS, verbose=VERBOSE, rect=RECT, cos_lr=COS_LR, optimizer=OPTIMIZER, patience=PATIENCE, exist_ok=EXIST_OK, overlap_mask=OVERLAP_MASK, mask_ratio=MASK_RATIO, fraction=FRACTION)
    except Exception as e:
        print(f'Error: {e}')

# --Inicio Menú
while True:
    try:
        model_type = input('Qué tipo de modelo desea utilizar, indique "original" o "entrenado": ')
        if model_type == "original":
            model_train = train(YOLO(model_orig))
            break  # Sale del bucle si la entrada es válida
        elif model_type == "entrenado":
            model_train = train(YOLO(model_marcas)) 
            break  # Sale del bucle si la entrada es válida
        else:
            print('Character not valid, try again. ')
    except Exception as e:
        print(f'Error when performing the training task: {e}')
# --Finalización Menú

New https://pypi.org/project/ultralytics/8.2.28 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.220 🚀 Python-3.10.13 torch-2.1.1 CPU (AMD Ryzen 5 4500U with Radeon Graphics)
engine\trainer: task=segment, mode=train, model=c:\Users\matrix\pruebayolo\proyecto_yolo\dist\model_version\yolov8n-seg.pt, data=c:\Users\matrix\pruebayolo\proyecto_yolo\dist\data_marks\dataset.yaml, epochs=45, patience=45, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=1, project=segmentation, name=training_marks2, exist_ok=False, pretrained=True, optimizer=auto, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=1, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=

train: Scanning C:\Users\matrix\pruebayolo\proyecto_yolo\dist\data_marks\YOLODataset_train\labels.cache... 497 images, 1 backgrounds, 0 corrupt: 100%|██████████| 498/498 [00:00<?, ?it/s]
val: Scanning C:\Users\matrix\pruebayolo\proyecto_yolo\dist\data_marks\YOLODataset_val\labels.cache... 150 images, 0 backgrounds, 0 corrupt: 100%|██████████| 150/150 [00:00<?, ?it/s]


Plotting labels to segmentation\training_marks2\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=4.9e-05, momentum=0.9) with parameter groups 66 weight(decay=0.0), 77 weight(decay=0.0005), 76 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to segmentation\training_marks2
Starting training for 45 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


       1/45         0G      1.532      3.108       6.87      1.158          6        640: 100%|██████████| 32/32 [05:23<00:00, 10.11s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:35<00:00,  7.19s/it]

                   all        150        284          0          0          0          0          0          0          0          0


...

                   all        150        284    0.00444      0.338     0.0127    0.00947    0.00385      0.305     0.0117    0.00647

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


      45/45         0G     0.9303      1.019      3.766     0.9169          3        640: 100%|██████████| 32/32 [05:20<00:00, 10.01s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:43<00:00,  8.76s/it]


                   all        150        284    0.00433      0.328     0.0127    0.00947    0.00381      0.305     0.0117    0.00651

45 epochs completed in 4.633 hours.
Optimizer stripped from segmentation\training_marks2\weights\last.pt, 7.5MB
Optimizer stripped from segmentation\training_marks2\weights\best.pt, 7.5MB

Validating segmentation\training_marks2\weights\best.pt...
Ultralytics YOLOv8.0.220 🚀 Python-3.10.13 torch-2.1.1 CPU (AMD Ryzen 5 4500U with Radeon Graphics)
YOLOv8n-seg summary (fused): 195 layers, 3624046 parameters, 0 gradients, 13.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:31<00:00,  6.37s/it]


                   all        150        284    0.00531      0.282     0.0157     0.0126    0.00531      0.282     0.0157    0.00887
Speed: 2.4ms preprocess, 145.3ms inference, 0.0ms loss, 4.1ms postprocess per image
Results saved to segmentation\training_marks2


<span style="font-family: 'Bahnschrift Light'; font-size: 40px;">Configuración para la exportación</span>

<span style="font-family: 'Bahnschrift Light'; font-size: 18px;">A continuación se muestra una tabla de referencia para exportar un modelo YOLOv8 entrenado en la tarea segmentación de instancias. Tenga en cuenta que configurar el parámetro de `format` es fundamental para el proceso de exportación. Antes de continuar, asegúrese de verificar y ajustar estos valores a sus requisitos específicos: </span>

| Formatos                                                             | Asignación | Extensión                     | Hyperparámetros                                            | Descripción                                        |
|--------------------------------------------------------------------|-------------------|---------------------------|-----------------------------------------------------|----------------------------------------------------|
| [PyTorch](https://pytorch.org/)                                    | -                 | `yolov8n.pt`              | -                                                   | Modelo en formato PyTorch                           |
| [TorchScript](https://pytorch.org/docs/stable/jit.html)            | "torchscript"     | `yolov8n.torchscript`     | `imgsz`, `optimize`                                 | Simplifica la implementación de modelos PyTorch en entornos de producción y aplicaciones eficientes, mejorando la portabilidad y el rendimiento al permitir ejecutar una representación intermedia en entornos sin Python.                       |
| [ONNX](https://onnx.ai/)                                           | "onnx"            | `yolov8n.onnx`            | `imgsz`, `half`, `dynamic`, `simplify`, `opset`     | Desarrollado para promover la interoperabilidad, la optimización del hardware y la colaboración entre comunidades, al tiempo que responde a la necesidad de portabilidad de los modelos entre distintos marcos y herramientas de aprendizaje automático.                              |
| [OpenVINO](https://docs.openvino.ai/latest/index.html)             | "openvino"        | `yolov8n_openvino_model/` | `imgsz`, `half`, `int8`                             | Elaborado para promover la interoperabilidad, la optimización del hardware y el despliegue eficiente de modelos a través de diferentes marcos y herramientas de aprendizaje automático, con especial atención a las plataformas de hardware Intel.                     |
| [TensorRT](https://developer.nvidia.com/tensorrt)                  | "engine"          | `yolov8n.engine`          | `imgsz`, `half`, `dynamic`, `simplify`, `workspace` | Permite promover la interoperabilidad, la optimización del hardware y la implantación eficiente de modelos en distintos marcos y herramientas de aprendizaje automático, con especial atención a las plataformas de hardware de NVIDIA.                     |
| [CoreML](https://github.com/apple/coremltools)                     | "coreml"          | `yolov8n.mlpackage`       | `imgsz`, `half`, `int8`, `nms`                      | Posibilita promover la interoperabilidad, la optimización del hardware y el despliegue eficiente de modelos a través de diferentes marcos y herramientas de aprendizaje automático, con especial atención a las plataformas de hardware de Apple.                            |
| [TF SavedModel](https://www.tensorflow.org/guide/saved_model)      | "saved_model"     | `yolov8n_saved_model/`    | `imgsz`, `keras`, `int8`                            | Empleado para guardar, compartir y desplegar modelos entrenados con TensorFlow. Versátil y facilita el despliegue en diversas plataformas como servidores, dispositivos móviles, embebidos y navegadores.                     |
| [TF Lite](https://www.tensorflow.org/lite)                         | "tflite"          | `yolov8n.tflite`          | `imgsz`, `half`, `int8`                             | Diseñado para el aprendizaje automático en dispositivos, TF Lite aborda restricciones clave como latencia, privacidad, conectividad, tamaño y consumo de energía. Es esencial para desplegar modelos en dispositivos móviles e integrados, ofreciendo una solución ligera y eficiente.                           |
| [TF Edge TPU](https://coral.ai/docs/edgetpu/models-intro/)         | "edgetpu"         | `yolov8n_edgetpu.tflite`  | `imgsz`                                             | Utilizado para desplegar modelos de aprendizaje automático en el Edge TPU de TensorFlow. El Edge TPU es un pequeño ASIC (Circuito Integrado Específico de Aplicación) diseñado por Google para ofrecer inferencias de aprendizaje automático de alto rendimiento en dispositivos de bajo consumo.                       |
| [TF.js](https://www.tensorflow.org/js)                             | "tfjs"            | `yolov8n_web_model/`      | `imgsz`                                             | Facilita el despliegue de modelos de aprendizaje automático en navegadores web y Node.js, destacando la portabilidad y la facilidad de uso.                    |
| [PaddlePaddle](https://github.com/PaddlePaddle)                    | "paddle"          | `yolov8n_paddle_model/`   | `imgsz`                                             | Utilizado para desplegar modelos en PaddlePaddle, una plataforma de aprendizaje profundo de código abierto, paralela y distribuida que tiene su origen en la práctica industrial.                      |
| [ncnn](https://github.com/Tencent/ncnn)                            | "ncnn"            | `yolov8n_ncnn_model/`     | `imgsz`, `half`                                     | Formato optimizado para plataformas móviles, ofreciendo alto rendimiento. Puede incluir una estructura de archivo de modelo con información sobre capas, blobs de entrada y salida, y otros parámetros.                              |


<span style="font-family: 'Bahnschrift Light'; font-size: 40px;">EXPORTACIÓN</span>


In [15]:
FORMAT='onnx' # Seleccione el formato de exportación del modelo segmentación, empleando la tabla previamente proporcionada; en la columna "Asignación" se indican las opciones para ajustar este valor.
INT8=True # Establezca este parámetro en True al utilizar la CPU y en False en caso contrario. La cuantificación a INT8 mejora la eficiencia del modelo segmentación en cuanto a memoria y velocidad de inferencia, especialmente en hardware que admite esta precisión.
HALF=False # Configúrelo en True cuando use la GPU; en caso contrario, False. La cuantificación a FP16 mejora la eficiencia de la memoria del modelo segmentación y la velocidad de inferencia, especialmente en hardware que admite precisión de punto flotante de 16 bits.
IMGSZ=640 # Establezca las dimensiones en píxeles de la imagen de entrada para la exportación del modelo de segmentación. Puede especificarlo como un número entero, por ejemplo, 640 para un cuadrado perfecto, o como una tupla, por ejemplo, (640, 480) para dimensiones específicas de ancho y alto. Las imágenes que ingreses al modelo después de la exportación deben tener las mismas dimensiones específicas que has configurado para adaptarse a los requisitos del escenario de despliegue.
OPTIMIZE=False # Controla la optimización en modelos de segmentación de instancias a TorchScript para su implementación móvil. Es importante destacar que esta función puede resultar en un aumento significativo en el tamaño del modelo exportado, lo cual puede no ser ideal para aplicaciones móviles. Se configura con True para activar y False para desactivar.
DYNAMIC=False # Controla la habilitación de ejes dinámicos en modelos de segmentación, lo cual es particularmente útil para gestionar tamaños de lote variables. Esta característica funciona bien en escenarios donde el tamaño del lote puede cambiar durante la inferencia, como aplicaciones en tiempo real o de transmisión por secuencias. Los valores aceptados son Verdadero para habilitar la función y Falso para deshabilitarla.
SIMPLIFY=False # En la exportación de modelos de segmentación a ONNX|TensorRT, este hiperparámetro personaliza la complejidad del modelo, optimizando, eliminando capas redundantes y reduciendo la precisión de los parámetros. Se activa con True y se desactiva con False.
OPSET=None # Especifica la versión del conjunto de operadores en ONNX al exportar el modelo segmentación desde marcos como PyTorch o TensorFlow. Si se deja en "None", ONNX utilizará automáticamente la versión más reciente disponible; para una versión específica, asigne el número entre comillas, por ejemplo, "11".
WORKSPACE=4 # En la exportación de modelos de segmentación a TensorRT, establece el tamaño del espacio de trabajo en GB asignado para optimizar y preparar el modelo de red neuronal. Este espacio se utiliza durante el proceso de construcción del motor para lograr una ejecución eficiente en hardware GPU mediante la biblioteca TensorRT.
NMS=False # En la exportación de modelos de segmentación a CoreML, controla la inclusión de la Supresión No Máxima (NMS) en el modelo para eliminar cuadros delimitadores redundantes en la segmentación de instancias y mejorar la precisión de las predicciones. Establecer 'NMS' en 'False' ignora NMS en los modelos CoreML exportados.  Este ajuste, configurable durante la exportación del modelo YOLO, lo que permite a los usuarios optimizar la implementación del modelo en una variedad de plataformas y dispositivos.
KERAS= False # En la exportación de modelos de segmentación a TF SavedModel y TF Lite, permite optimizar el despliegue en diversas plataformas y dispositivos. Incluye también el formato del archivo, el dispositivo de ejecución y la posibilidad de manejar múltiples etiquetas por caja. Establezca el valor del hiperparámetro en True si está familiarizado con Keras; de lo contrario, en False para excluir su uso en la exportación.

def export(model_export):
    try:
        model_export = YOLO(model_export)
        model_export.export(format=FORMAT, imgsz=IMGSZ, dynamic=DYNAMIC, opset=OPSET, workspace=WORKSPACE, nms=NMS, keras=KERAS, int8=INT8, half=HALF, optimize=OPTIMIZE, simplify=SIMPLIFY)
    except Exception as e:
        print(f'Error in the export of the model: {e}') 

export(model_marcas)

Ultralytics YOLOv8.0.220 🚀 Python-3.10.13 torch-2.1.1 CPU (AMD Ryzen 5 4500U with Radeon Graphics)
YOLOv8n-seg summary (fused): 195 layers, 3266059 parameters, 0 gradients, 12.0 GFLOPs

PyTorch: starting from 'c:\Users\matrix\pruebayolo\proyecto_yolo\src\srcv5\model_marks\segmentation\training_marks\weights\best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) ((1, 77, 8400), (1, 32, 160, 160)) (6.5 MB)

ONNX: starting export with onnx 1.15.0 opset 17...
ONNX: export success ✅ 1.8s, saved as 'c:\Users\matrix\pruebayolo\proyecto_yolo\src\srcv5\model_marks\segmentation\training_marks\weights\best.onnx' (12.7 MB)

Export complete (4.4s)
Results saved to C:\Users\matrix\pruebayolo\proyecto_yolo\src\srcv5\model_marks\segmentation\training_marks\weights
Predict:         yolo predict task=segment model=c:\Users\matrix\pruebayolo\proyecto_yolo\src\srcv5\model_marks\segmentation\training_marks\weights\best.onnx imgsz=640 int8 
Validate:        yolo val task=segment model=c:\Users\

<span style="font-family: 'Bahnschrift Light'; font-size: 40px;">VALIDACIÓN</span>


In [25]:
NAME='validation_marks' # El nombre del experimento de validación para segmentación de instancias deben ir entre comillas. Evite el uso de espacios al nombrar las carpetas; en su lugar, utilice algún formato de nombres como camelCase, snake_case o PascalCase.
DATA= data_train_marks # Permite indicar la ruta al archivo que contiene los metadatos necesarios para el proceso de validación, la ruta debe ser proporcionada entre comillas.
SAVE_JSON=True # Si se configura como True, habilita la funcionalidad de guardar los resultados obtenidos de manera detallada del proceso de validación en un formato estructurado JSON.
IMGSZ=640 # Establece las dimensiones en píxeles de la imagen de entrada para la validación del modelo de segmentación. Puede ser un número entero, como 640 para un cuadrado perfecto, o una tupla, como (640, 480), para dimensiones específicas de ancho y alto. Se recomienda usar el mismo valor que utilizo durante el entrenamiento del modelo.
BATCH=16 # Define la cantidad de imágenes procesadas simultáneamente en una iteración para la validación de segmentos. El valor predeterminado es 16; se recomienda establecerlo en -1 para aprovechar AutoBatch, que ajusta automáticamente el tamaño del lote para optimizar el rendimiento, evitar problemas de memoria y maximizar la eficiencia del entrenamiento. Si desea personalizarlo, exprese el valor del parámetro como un número entero.
SAVE_HYBRID=True # Activa la función con True para guardar una versión híbrida de la etiqueta, incluyendo la original y predicciones adicionales. Útil para el análisis detallado del rendimiento del modelo segmentación durante la validación; establezca en False para mostrar solo las predicciones.
CONF=0.5 # Establece el umbral de confianza para la validación de clases en la tarea de segmentación. Se recomienda un valor entre 0.5 y 0.10. Un umbral más alto mejora la precisión pero reduce la frecuencia de predicciones, mientras que un umbral más bajo aumenta la frecuencia pero disminuye la precisión. 
MAX_DET=10 # Toma como valor solo números enteros. Índica el límite de la cantidad máxima de objetos que el modelo intentara segmentar en una imagen. Se recomienda establecer un valor alto para evitar perder detecciones relevantes.
DEVICE='CPU' # Especifica el dispositivo de ejecución para la prueba de validación en la operación de segmentación. Puede seleccionar entre CPU o GPU. Si no dispone de una GPU con Cuda, se recomienda utilizar la CPU mediante el parámetro device='cpu'. En caso de contar con Cuda, puede especificar una GPU con device='cuda:0'; el número representa el identificador de la GPU disponible en el sistema. También es posible utilizar múltiples GPUs mediante device='cuda:0,1,2'.
PLOTS=True # Utilice valores booleanos (Verdadero o Falso) para controlar la generación de gráficos que permite visualizar y monitorear la pérdida y la precisión durante la validación en la segmentación de instancias. Establecer plots=True activara la función; si desea desactivarla, establezca el valor del hiperparámetro en False.
RECT=False # Habilita la formación rectangular en cada lote, redimensionando las imágenes para que todas tengan la misma forma rectangular. Puedes establecerlo en True si tu conjunto de datos es extenso y deseas acelerar el tiempo de validación en la segmentación de instancias. De lo contrario, si se establece en False el modelo se entrena en el orden normal procesando todos los datos de un lote antes de pasar al siguiente.
IOU=0.6 # El umbral predeterminado para la supresión no máxima (NMS) en la validación YOLO es 0,6. Este umbral de IoU (intersección sobre unión) es fundamental para NMS porque determina el grado mínimo de superposición requerido para que dos cuadros delimitadores se consideren el mismo segmento. Un umbral de IoU más bajo hace que NMS sea más conservador, mientras que un umbral de IoU más alto permite que un NMS más relajado evite eliminar los verdaderos positivos.

def val(model_val):
    try:
        model_val.val(name=NAME, data=DATA, save_json=SAVE_JSON, imgsz=IMGSZ, batch=BATCH, save_hybrid=SAVE_HYBRID, conf=CONF, max_det=MAX_DET, device=DEVICE, plots=PLOTS, rect=RECT, iou=IOU)
    except Exception as e:
        print(f'Error: {e}')

# --Inicio Menú
while True:
    try:
        model_type = input('Qué tipo de modelo desea utilizar, indique "original" o "entrenado": ')
        if model_type == "original":
            val(YOLO(model_orig))
            break  # Sale del bucle si la entrada es válida
        elif model_type == "entrenado":
            val(YOLO(model_marcas))
            break  # Sale del bucle si la entrada es válida
        else:
            print('Character not valid, try again. ')
    except Exception as e:
        print(f'Error when performing the validation task: {e}')
# --Finalización Menú

Ultralytics YOLOv8.0.220 🚀 Python-3.10.13 torch-2.1.1 CPU (AMD Ryzen 5 4500U with Radeon Graphics)
YOLOv8n-seg summary (fused): 195 layers, 3266059 parameters, 0 gradients, 12.0 GFLOPs


val: Scanning C:\Users\matrix\pruebayolo\proyecto_yolo\dist\data_chevrolet_toyota\val\labels... 32 images, 0 backgrounds, 0 corrupt: 100%|██████████| 32/32 [00:00<00:00, 293.57it/s]

val: New cache created: C:\Users\matrix\pruebayolo\proyecto_yolo\dist\data_chevrolet_toyota\val\labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:05<00:00,  2.77s/it]


                   all         32         61          1          1      0.995      0.995          0          0          0          0
               captiva         32          5          1          1      0.995      0.995          0          0          0          0
                  beat         32          5          1          1      0.995      0.995          0          0          0          0
                 tahoe         32          7          1          1      0.995      0.995          0          0          0          0
                tacoma         32          2          1          1      0.995      0.995          0          0          0          0
                 hilux         32          1          1          1      0.995      0.995          0          0          0          0
             chevrolet         32          1          1          1      0.995      0.995          0          0          0          0
               sequoia         32         11          1          1   

<span style="font-family: 'Bahnschrift Light'; font-size: 40px">Configuración de fuentes</span>

<span style="font-family: 'Bahnschrift Light'; font-size: 18px">Para utilizar múltiples fuentes de datos al realizar predicciones con el modelo, se requiere que se ajuste el parámetro'source' a sus necesidades, tal como se indica en la siguiente tabla:</span>


| Fuentes          | Asignación                             | Tipo             | Notas                                                           |
| --------------- | ------------------------------------ | ----------------- | --------------------------------------------------------------- |
| `image`           | 'image.jpg'                          | str or Path       | Archivo que contiene una única imagen.                                              |
| `URL`             | 'https://ultralytics.com/images/bus.jpg' | str               | Dirección que especifica la ubicación de una imagen en la web.                                                 |
| `screenshot`      | 'screen'                             | str               | El sistema captura la imagen actualmente visible en la pantalla y la utiliza como entrada para el modelo.                                           |
| `PIL`             | Image.open('im.jpg')                 | PIL.Image         | Utilizado para cargar imágenes en formato HWC (altura, ancho, canales) con canales RGB (rojo, verde y azul) mediante la biblioteca Python Imaging Library (PIL).                                   |
| `OpenCV`          | cv2.imread('im.jpg')                 | np.ndarray        | Permite la lectura de una imagen desde un archivo en formato HWC con canales BGR (azul, verde, rojo) utilizando la biblioteca OpenCV, almacenando la imagen como un array de NumPy.                    |
| `numpy`           | np.zeros((640,1280,3))               | np.ndarray        | Genera un array de ceros con las dimensiones especificadas para un formato HWC con canales BGR, utilizando la biblioteca NumPy.                    |
| `torch`           | torch.zeros(16,3,320,640)            | torch.Tensor      | Crea un tensor de ceros con las dimensiones especificadas para un formato HWC con canales RGB, empleando el framework PyTorch.               |
| `CSV`             | 'sources.csv'                        | str or Path       | Archivo de texto que almacena las rutas a las imágenes que se procesarán.   |
| `video`          | 'video.mp4'                          | str or Path       | Proporciona acceso a un archivo de video único.                       |
| `directory`      | 'path/'                              | str or Path       | Directorio que contiene múltiples archivos de imagen.               |
| `glob`           | 'path/*.jpg'                         | str               | Permite acceder a varias imágenes en un directorio usando expresiones de coincidencia de patrones. |
| `YouTube`        | 'https://youtu.be/LNwODJXcvt4'       | str               | Facilita el acceso a videos desde la plataforma YouTube.                                         |
| `stream`         | 'rtsp://example.com/media.mp4'      | str               | Permite la conexión a flujos de video o audio en tiempo real mediante protocolos como RTSP, RTMP, TCP o IP, ya sea a través de internet o una red local. |
| `multi-stream`   | 'list.streams'                       | str or Path       | Se utiliza para transmitir varios flujos de medios simultáneamente, permitiendo el procesamiento y análisis paralelo de múltiples flujos de medios. |

<span style="font-family: 'Bahnschrift Light'; font-size: 18px">Formatos para las imágenes: </span>

| Image Suffixes | Reference                           |
| --------------- | ----------------------------------- |
| .bmp            | [Microsoft BMP File Format](https://docs.fileformat.com/es/image/bmp/)           |
| .dng            | [Adobe DNG](https://docs.fileformat.com/es/image/dng/)                           |
| .jpeg           | [JPEG](https://docs.fileformat.com/es/image/jpeg/)                                |
| .jpg            | [JPEG](https://docs.fileformat.com/es/image/jpeg/)                                |
| .mpo            | [Multi Picture Object](https://docs.fileformat.com/es/image/mpo/)                |
| .png            | [Portable Network Graphics](https://docs.fileformat.com/es/image/png/)           |
| .tif            | [Tag Image File Format](https://docs.fileformat.com/es/image/tiff/)               |
| .tiff           | [Tag Image File Format](https://docs.fileformat.com/es/image/tiff/)               |
| .webp           | [WebP](https://docs.fileformat.com/es/image/webp/)                                |
| .pfm            | [Portable FloatMap](https://docs.fileformat.com/font/pfm/)                   |


<span style="font-family: 'Bahnschrift Light'; font-size: 18px">Formatos para los videos: </span>

| Video Suffixes | Reference                           |
| -------------- | ----------------------------------- |
| .asf           | [Advanced Systems Format](https://docs.fileformat.com/es/video/asf/)             |
| .avi           | [Audio Video Interleave](https://docs.fileformat.com/es/video/avi/)              |
| .gif           | [Graphics Interchange Format]()          |
| .m4v           | [MPEG-4 Part 14](https://docs.fileformat.com/es/video/m4v/)                      |
| .mkv           | [Matroska](https://docs.fileformat.com/es/video/mkv/)                            |
| .mov           | [QuickTime File Format](https://docs.fileformat.com/es/video/mov/)               |
| .mp4           | [MPEG-4](https://docs.fileformat.com/es/video/mp4/)          |
| .mpeg          | [MPEG-1](https://docs.fileformat.com/es/video/mpeg/)                       |
| .mpg           | [MPEG-1](https://docs.fileformat.com/es/video/mpeg/)                       |
| .ts            | [MPEG Transport Stream](https://docs.fileformat.com/es/video/ts/)               |
| .wmv           | [Windows Media Video](https://docs.fileformat.com/es/video/wmv/)                 |
| .webm          | [WebM Project](https://docs.fileformat.com/es/video/webm/)                        |


<span style="font-family: 'Bahnschrift Light'; font-size: 40px">PREDICCIÓN</span>


In [34]:
SOURCE= dir_prueba # Establezca el origen de datos que el modelo de detección utilizará para realizar predicciones. Configure el valor de este hiperparámetro según la tabla proporcionada anteriormente.
MAX_DET=10 # Toma como valor solo números enteros. Índica el límite de la cantidad máxima de objetos que el modelo intentara predecir en una imagen. Se recomienda establecer un valor alto para evitar perder detecciones relevantes.
IMGSZ=640 # Establezca las dimensiones en píxeles de la imagen de entrada durante la predicción en tareas de detección. Puede ser un número entero, como 640 para un cuadrado perfecto, o una tupla, como (640, 480), para dimensiones específicas de ancho y alto. Se recomienda utilizar los mismos valores utilizados durante el entrenamiento del modelo para mantener la coherencia en la inferencia.
CONF=0.34 # Establece el umbral de confianza durante el proceso de predicción en la tarea de detección. Se recomienda establecer el valor hiperparámetro entre 0.5 y 0.10. Un umbral más alto mejora la precisión pero reduce la frecuencia de predicciones, mientras que un umbral más bajo aumenta la frecuencia pero disminuye la precisión en la inferencia.
LINE_WIDTH= None # Determina el grosor en píxeles de los cuadros delimitadores que rodean los objetos detectados por el modelo. Puede establecer el grosor de la línea como un número entero en el que, a mayor valor, la línea será más gruesa, también puede utilizar como valor None para que el grosor se ajuste de forma automatizada, proporcionando una línea proporcional al tamaño de la imagen.
VISUALIZE=False # Determina si las características del modelo de detección deben mostrarse durante la predicción. Establecer esto en True permite que las características se muestren como mapas intermedias, lo que hace que el modelo sea más fácil de entender. Si se establece en False, no se mostrarán las características del modelo. 
IOU=0.5 # El umbral predeterminado para la supresión no máxima (NMS) en la validación YOLO es 0,7. Este umbral de IoU (intersección sobre unión) es fundamental para NMS porque determina el grado mínimo de superposición requerido para que dos cuadros delimitadores se consideren la misma detección. Un umbral de IoU más bajo hace que NMS sea más conservador, mientras que un umbral de IoU más alto permite que un NMS más relajado evite eliminar los verdaderos positivos.
DEVICE='cpu' # Especifica el dispositivo de ejecución para la prueba de predicción en la operación de detección. Puede seleccionar entre CPU o GPU. Si no dispone de una GPU con Cuda, se recomienda utilizar la CPU mediante el parámetro device='cpu'. En caso de contar con Cuda, puede especificar una GPU con device='cuda:0'; el número representa el identificador de la GPU disponible en el sistema. También es posible utilizar múltiples GPUs mediante device='cuda:0,1,2'.
VID_STRIDE=False # Controla la velocidad de los fotogramas durante el proceso de predicción en vídeos o secuencias de tiempo real. Al establecerlo en True el modelo se adapta a la velocidad de fotogramas especificada por la fuente de vídeo, procesando cada fotograma individualmente. Para desactivar esta función indique como valor False.
STREAM_BUFFER=False # Controla el almacenamiento en búfer de los fotogramas para la detección. Si es True, se almacenan todos los fotogramas para el procesamiento en tiempo real de vídeos o transmisiones en directo; si es False, devuelve el fotograma más reciente.
SAVE_FRAMES=False # Controla la captura y almacenamiento de los fotogramas predichos por el modelo de detección. Con True, se guardarán todos los fotogramas individuales predichos; con False, no se realizará el almacenamiento de los fotogramas.
AUGMENT=False # Aplica transformaciones a las imágenes de entrada, tales como giros, rotaciones, recortes y cambios de color, para diversificar los datos y mejorar la predicción en la detección. Establecer en True para activar la función, False para desactivar.
SAVE_CROP=True # Determina si se deben guardar imágenes recortadas con los resultados durante la predicción en la detección. Al establecerlo en "False", las imágenes recortadas no se guardarán, lo que reduce el tamaño del archivo. Con el valor "True", se guardarán las imágenes recortadas correspondientes a las áreas detectadas.
SHOW=False # Determina si se deben mostrar las imágenes o vídeos detectados durante la predicción. Al establecerlo en "True", permite la visualización de las predicciones en el mismo entorno, proporcionando una representación visual de los resultados. Si se establece en "False", las predicciones no se mostrarán. 
CLASSES= [30, 5, 11] # Permite definir las clases de objetos que el modelo está diseñado para detectar. Si se desean segmentar múltiples clases, se deben especificar los índices correspondientes a cada clase entre corchetes [], separados por comas. En caso de que no se desee segmentar clases específicas, se puede proporcionar el número total de clases existentes como índice.
CLASSES= 41
SAVE= True # Cuando se establece en True, el modelo guarda los puntos de control periódicamente durante la predicción. 
SAVE_CONF=True # Permite guardar los resultados de la segmentación en un archivo de texto con un formato [clase], [valor_x], [valor_y], [ancho], [alto] y [confianza].
SAVE_TXT= True
NAME_PREDICTION='predict_marks'

def thread_safe_predict(model): # Realiza una predicción segura utilizando un modelo YOLO.
    try:
        local_model = YOLO(model) # Inicializa el modelo YOLO.
        
        results= local_model.predict(source=SOURCE, save=SAVE, classes=CLASSES, conf=CONF, name=NAME_PREDICTION, save_txt=SAVE_TXT, max_det=MAX_DET, line_width=LINE_WIDTH, visualize=VISUALIZE, imgsz=IMGSZ, iou=IOU, device=DEVICE, vid_stride=VID_STRIDE, stream_buffer=STREAM_BUFFER, save_crop=SAVE_CROP, show=SHOW, save_frames=SAVE_FRAMES, save_conf=SAVE_CONF, retina_masks=True, agnostic_nms=True) # Se establecen los hiperpametros a utilizar en el proceso de predicción.
        output_queue.put(results)
        
    except Exception as e:
        print(f"Error when executing the prediction task: {e}")
        
output_queue = Queue() # Creación de una cola para comunicación segura con hilos.
try: 
    Thread(target=thread_safe_predict, args=(model_marcas,)).start() # Inicio de un nuevo hilo para la predicción del coche.
    output_queue.get() # Obtiene los resultados de la cola.
except Exception as e: # Captura de excepciones para la predicción de marcas.
    print(f"Error in prediction by: {e} ") 



image 1/20 c:\Users\matrix\pruebayolo\proyecto_yolo\src\assets_marks\D_NQ_NP_2X_607554-MCO74306558674_022024-F.jpg: 608x640 1 chevrolet, 146.0ms
image 2/20 c:\Users\matrix\pruebayolo\proyecto_yolo\src\assets_marks\D_NQ_NP_2X_625969-MCO74109549689_012024-F.jpg: 544x640 1 chevrolet, 111.0ms
image 3/20 c:\Users\matrix\pruebayolo\proyecto_yolo\src\assets_marks\D_NQ_NP_2X_634781-MCO74716081265_022024-F.jpg: 512x640 1 chevrolet, 1 onix, 106.0ms
image 4/20 c:\Users\matrix\pruebayolo\proyecto_yolo\src\assets_marks\D_NQ_NP_2X_651757-MCO74531359442_022024-F.jpg: 480x640 1 toyota, 103.0ms
image 5/20 c:\Users\matrix\pruebayolo\proyecto_yolo\src\assets_marks\D_NQ_NP_2X_697181-MCO73688884097_122023-F.jpg: 512x640 1 chevrolet, 103.0ms
image 6/20 c:\Users\matrix\pruebayolo\proyecto_yolo\src\assets_marks\D_NQ_NP_2X_734517-MCO74482617882_022024-F.jpg: 480x640 1 toyota, 101.0ms
image 7/20 c:\Users\matrix\pruebayolo\proyecto_yolo\src\assets_marks\D_NQ_NP_2X_765472-MCO74484798046_022024-F.jpg: 480x640 1 t